### Final Results With Comparison to Different Model Performance

In [1]:
!pip install xgboost


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
print(pd.__version__)
import numpy as np
import json
import logging
import xgboost as xgb
import ast
import itertools
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2
import os
import sys

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

2.1.3


In [3]:
sys.path.append('/root/Unified_LN_V2/')
from Utilities import *

In [4]:
def air_calc_new(pred,prefix):
    air = {}
    target_control_dict = [
            {'age_cuts': {
                'target': ['Age 62 and over'],
                'control': ['Age below 62']
            }},
            {'Race': {
                'target': ['black', 'hispanic'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['black'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['hispanic'],
                'control': ['white']
            }},
            {'Gender': {
                'target': ['F'],
                'control': ['M']
            }},
            # 'mmct_tag': {
            #     'target': ['mmct'],
            #     'control': ['non-mmct']
            # }
    ]

    for var in target_control_dict:
        for i in var:
            target_list = var[i]['target']
            control_list = var[i]['control']
            p_target_list= ','.join(target_list) if (
                (len(target_list)<= 1) and (('black' in target_list) or ('hispanic' in target_list))
            ) else ''
            
            ttd_target= pred[pred[i].isin(target_list)]['con_wt'].sum()
            approved_target= pred[pred[i].isin(target_list)]['approved'].multiply(pred[pred[i].isin(target_list)]['con_wt']).sum()
            ttd_control= pred[pred[i].isin(control_list)]['con_wt'].sum()
            approved_control= pred[pred[i].isin(control_list)]['approved'].multiply(pred[pred[i].isin(control_list)]['con_wt']).sum()
            app_pct_target= round(approved_target/ttd_target, 4)
            app_pct_control= round(approved_control/ttd_control, 4)
            air[prefix+i+p_target_list]= [round(app_pct_target/app_pct_control, 4)]
        
            # print({'variable': i, 'target_list': p_target_list, 'ttd_target': ttd_target, 'ttd_control': ttd_control, 
                   # 'approved_target': approved_target, 'approved_control': approved_control})
            
    # print(air)
        
    return air

In [5]:
col_tuple_pass= ["unique_id", 'client', "wt2", 'fico_pb', 'auto_score', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts']

In [7]:
def get_risk_rank(data, target, cols_tuple, score_col= 'p1', weight= 'RECWGT', asc= False):
    dd = data.loc[:, cols_tuple + [score_col]]
    # dd = pd.DataFrame(dd)
    # dd = dd.reset_index()
    dd['RECWGT'] = dd["wt2"]
    dd['conwt'] = 1
    dd[target] = dd[target] 
    # pred = pd.concat([dd.loc[:, cols_tuple + ['RECWGT', 'conwt']],p],axis=1)
    pred= dd.copy()
    # display(pred.head())
    print(pred[weight].sum(),((pred[weight]*pred[target]).sum()),pred[weight].max())

    data_ks_cap5= rnkOrderingN(pred, target, weight, score_col, 5, asc)
    # display(data_ks_cap5[0])
    # print(data_ks_cap5[1])
    # data_ks_cap10= rnkOrderingN(pred, target, weight, score_col, 10, asc)
    # display(data_ks_cap10[0])
    # print(data_ks_cap10[1])
    return data_ks_cap5

In [8]:
def get_segment(d):
    if (d['derogcount']> 0):
        return 'Derog'
    elif ((d['derogcount']== 0) and (d['assetprop']> 0)):
        return 'Property Owner'
    elif ((d['derogcount']== 0) and (d['assetprop']== 0)):
        return 'Non Property Owner'
    else:
        return 'Missing'

In [9]:
def Equal_Approval_Cutoff_Barsath(pred, target, score_col, asc= True):
    pred= pred.copy()
    actual_approved_count= pred[target].sum()
    pred= pred.sort_values(by= [score_col], ascending= asc)
    pred['con_wt']= 1
    pred['cumsum']= pred['con_wt'].cumsum()
    pred['approved']= np.where(pred['cumsum']<= actual_approved_count, 1, 0)
    return pred #cutoff, original_dataset

## Reading Scored Data - Overall LDA

In [10]:
path_overall= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/With_LDA/Iteration_3/' #/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/With_LDA/
os.listdir(path_overall)

['With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 '050424_LN_Auto_KGB_LDA_No_Segment_Hopt_4.csv',
 'With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'xgbm_ln_auto_v2_hpt_best_with_lda_3.zip',
 'With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv']

In [11]:
# train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv")

# train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-02.csv")

train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv")
test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv")
oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv")
ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv")

train_scored_all.shape, test_scored_all.shape, oot_scored_all.shape, ttd_scored_all.shape

((146855, 12), (62938, 12), (37121, 12), (685215, 12))

In [12]:
funded_scored_all= pd.concat([train_scored_all, test_scored_all, oot_scored_all]).reset_index(drop= True)
funded_scored_all.shape, ttd_scored_all.shape, ttd_scored_all['funded_flag'].sum()

((246914, 12), (685215, 12), 200148.0)

In [13]:
funded_scored_all['unique_id']= funded_scored_all['unique_id'].astype(str)
ttd_scored_all['unique_id']= ttd_scored_all['unique_id'].astype(str)

In [14]:
funded_scored_all.rename(columns= {'p1': 'p1_overall_lda'}, inplace= True)
ttd_scored_all.rename(columns= {'p1': 'p1_overall_lda'}, inplace= True)

### Reading V 1.5 Scored Data

In [15]:
v15= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/040424_Auto_LN_V1_5_Scored_Data.csv')
v15.head()

,unique_id,client,p1_v1_5,fico_pb,dataset
0,453208,SECU,0.053346,624.0,fico_lt_660
1,453394,SECU,0.064319,0.0,fico_lt_660
2,453515,SECU,0.086988,614.0,fico_lt_660
3,453555,SECU,0.035332,637.0,fico_lt_660
4,453594,SECU,0.094630,587.0,fico_lt_660


In [16]:
v15.shape, ttd_scored_all.shape, funded_scored_all.shape

((824212, 5), (685215, 12), (246914, 12))

In [17]:
v15['unique_id']= v15['unique_id'].astype(str)

In [18]:
print(ttd_scored_all.shape)
ttd_scored_all= ttd_scored_all.merge(v15[['unique_id', 'client', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
ttd_scored_all.shape

(685215, 12)


(685215, 13)

In [19]:
ttd_scored_all['p1_v1_5'].isnull().sum()

0

In [20]:
ttd_scored_all[['p1_v1_5', 'p1_overall_lda']].describe()

,p1_v1_5,p1_overall_lda
count,685215.000000,685215.000000
mean,0.038407,0.021863
std,0.032355,0.022076
min,0.007456,0.005289
25%,0.014268,0.007273
50%,0.025966,0.015229
75%,0.052640,0.024805
max,0.190158,0.114281


In [21]:
print(funded_scored_all.shape)
funded_scored_all= funded_scored_all.merge(v15[['unique_id', 'client', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
funded_scored_all.shape, funded_scored_all['p1_v1_5'].isnull().sum()

(246914, 12)


((246914, 13), 0)

In [22]:
# rnkOrderingN(funded_scored_all[~(funded_scored_all['fico_pb']>= 740)], 'bad_flag', None, 'p1_overall_lda', 5)

In [23]:
# rnkOrderingN(funded_scored_all[~(funded_scored_all['fico_pb']>= 740)], 'bad_flag', None, 'p1_v1_5', 5)

#### Reading Property Owner Dataset

In [24]:
path_owner= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Property_Owner_Model/'
os.listdir(path_owner)

['Mono_Data',
 'Data',
 'Property_Owner_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'Property_Owner_280324_LN_Auto_KGB_LDA_Hopt_1.csv',
 'Property_Owner_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'xgbm_ln_auto_v2_hpt_best_1_Property_Owner.zip',
 'Property_Owner_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'Property_Owner_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv']

In [25]:
train_scored_owner= pd.read_csv(f'{path_owner}Property_Owner_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv')
test_scored_owner= pd.read_csv(f'{path_owner}Property_Owner_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv')
oot_scored_owner= pd.read_csv(f'{path_owner}Property_Owner_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv')
ttd_scored_owner= pd.read_csv(f'{path_owner}Property_Owner_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv')
train_scored_owner.shape, test_scored_owner.shape, oot_scored_owner.shape, ttd_scored_owner.shape

((82033, 12), (35158, 12), (20750, 12), (303020, 12))

In [26]:
train_scored_owner['unique_id']= train_scored_owner['unique_id'].astype(str)
test_scored_owner['unique_id']= test_scored_owner['unique_id'].astype(str)
oot_scored_owner['unique_id']= oot_scored_owner['unique_id'].astype(str)
ttd_scored_owner['unique_id']= ttd_scored_owner['unique_id'].astype(str)

In [27]:
train_scored_owner['dataset']= 'Property Owner'
test_scored_owner['dataset']= 'Property Owner'
oot_scored_owner['dataset']= 'Property Owner'
ttd_scored_owner['dataset']= 'Property Owner'

#### Reading Non Property Owner Dataset

In [28]:
path_non_owner= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Non_Property_Owner_Model/'
os.listdir(path_non_owner)

['Non_Property_Owner_290324_LN_Auto_KGB_LDA_Hopt_1.csv',
 'Mono_Data',
 'Data',
 'Non_Property_Owner_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'Non_Property_Owner_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'Non_Property_Owner_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'xgbm_ln_auto_v2_hpt_best_1_Non_Property_Owner.zip',
 'Non_Property_Owner_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv']

In [29]:
train_scored_nonowner= pd.read_csv(f'{path_non_owner}Non_Property_Owner_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv')
test_scored_nonowner= pd.read_csv(f'{path_non_owner}Non_Property_Owner_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv')
oot_scored_nonowner= pd.read_csv(f'{path_non_owner}Non_Property_Owner_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv')
ttd_scored_nonowner= pd.read_csv(f'{path_non_owner}Non_Property_Owner_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv')
train_scored_nonowner.shape, test_scored_nonowner.shape, oot_scored_nonowner.shape, ttd_scored_nonowner.shape

((48949, 12), (20979, 12), (12460, 12), (275182, 12))

In [30]:
train_scored_nonowner['unique_id']= train_scored_nonowner['unique_id'].astype(str)
test_scored_nonowner['unique_id']= test_scored_nonowner['unique_id'].astype(str)
oot_scored_nonowner['unique_id']= oot_scored_nonowner['unique_id'].astype(str)
ttd_scored_nonowner['unique_id']= ttd_scored_nonowner['unique_id'].astype(str)

In [31]:
train_scored_nonowner['dataset']= 'Non Property Owner'
test_scored_nonowner['dataset']= 'Non Property Owner'
oot_scored_nonowner['dataset']= 'Non Property Owner'
ttd_scored_nonowner['dataset']= 'Non Property Owner'

In [32]:
train_ono= pd.concat([train_scored_owner, train_scored_nonowner]).reset_index(drop= True)
test_ono= pd.concat([test_scored_owner, test_scored_nonowner]).reset_index(drop= True)
oot_ono= pd.concat([oot_scored_owner, oot_scored_nonowner]).reset_index(drop= True)
ttd_ono= pd.concat([ttd_scored_owner, ttd_scored_nonowner]).reset_index(drop= True)
train_ono.shape, test_ono.shape, oot_ono.shape, ttd_ono.shape

((130982, 13), (56137, 13), (33210, 13), (578202, 13))

In [33]:
print(train_ono['dataset'].value_counts(dropna= False), train_scored_owner.shape, train_scored_nonowner.shape)
print(test_ono['dataset'].value_counts(dropna= False), test_scored_owner.shape, test_scored_nonowner.shape)
print(oot_ono['dataset'].value_counts(dropna= False), oot_scored_owner.shape, oot_scored_nonowner.shape)
print(ttd_ono['dataset'].value_counts(dropna= False), ttd_scored_owner.shape, ttd_scored_nonowner.shape)

dataset
Property Owner        82033
Non Property Owner    48949
Name: count, dtype: int64 (82033, 13) (48949, 13)
dataset
Property Owner        35158
Non Property Owner    20979
Name: count, dtype: int64 (35158, 13) (20979, 13)
dataset
Property Owner        20750
Non Property Owner    12460
Name: count, dtype: int64 (20750, 13) (12460, 13)
dataset
Property Owner        303020
Non Property Owner    275182
Name: count, dtype: int64 (303020, 13) (275182, 13)


In [34]:
del train_scored_all, train_scored_nonowner, train_scored_owner
del test_scored_all, test_scored_nonowner, test_scored_owner
del oot_scored_all, oot_scored_nonowner, oot_scored_owner
del ttd_scored_nonowner, ttd_scored_owner

In [35]:
import gc
gc.collect()

0

### Merging p1-overall

In [36]:
print(train_ono.shape)
train_ono= train_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(train_ono.shape)
print(train_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(test_ono.shape)
test_ono= test_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(test_ono.shape)
print(test_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(oot_ono.shape)
oot_ono= oot_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(oot_ono.shape)
print(oot_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(ttd_ono.shape)
ttd_ono= ttd_ono.merge(ttd_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(ttd_ono.shape)
print(ttd_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

(130982, 13)
(130982, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(56137, 13)
(56137, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(33210, 13)
(33210, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(578202, 13)
(578202, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64


In [37]:
ttd_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), train_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), test_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), oot_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe()

(                  p1  p1_overall_lda        p1_v1_5
 count  578202.000000   578202.000000  578202.000000
 mean        0.019049        0.020840       0.035851
 std         0.021964        0.021242       0.031272
 min         0.002785        0.005289       0.007533
 25%         0.005432        0.007184       0.013489
 50%         0.010780        0.014842       0.023367
 75%         0.023258        0.024273       0.048259
 max         0.241591        0.114281       0.189014,
                   p1  p1_overall_lda        p1_v1_5
 count  130982.000000   130982.000000  130982.000000
 mean        0.014259        0.016182       0.025513
 std         0.017073        0.016741       0.023222
 min         0.002785        0.005289       0.007533
 25%         0.004748        0.006554       0.011764
 50%         0.007869        0.009230       0.016667
 75%         0.017256        0.019551       0.028584
 max         0.231375        0.114281       0.188945,
                  p1  p1_overall_lda       p

In [38]:
train_ono.groupby(['dataset'])[['p1', 'p1_overall_lda', 'p1_v1_5']].apply(lambda x: x.describe()).T

dataset        Non Property Owner                                          \
                            count      mean       std       min       25%   
p1                        48949.0  0.027368  0.021877  0.006297  0.014157   
p1_overall_lda            48949.0  0.028912  0.021202  0.005857  0.016469   
p1_v1_5                   48949.0  0.042286  0.029202  0.008874  0.021910   

dataset                                      Property Owner            \
                     50%       75%       max          count      mean   
p1              0.020486  0.031677  0.231375        82033.0  0.006437   
p1_overall_lda  0.021575  0.030206  0.114281        82033.0  0.008587   
p1_v1_5         0.031416  0.054241  0.188945        82033.0  0.015505   

dataset                                                                     
                     std       min       25%       50%       75%       max  
p1              0.004016  0.002785  0.003810  0.005333  0.007432  0.056321  
p1_overall_lda  0.004991  0.005289  0.005875  0.007080  0.008980  0.048399  
p1_v1_5         0.009173  0.007533  0.010352  0.012982  0.016703  0.112125

In [39]:
train_ono[train_ono['dataset'].isin(['Property Owner'])].shape

(82033, 15)

In [40]:
train_ono['dataset'].value_counts(dropna= False)

dataset
Property Owner        82033
Non Property Owner    48949
Name: count, dtype: int64

In [41]:
def get_model_comparison_segment(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a

In [42]:
def get_model_comparison_overall(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1_overall_lda'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a
    

In [43]:
def get_model_comparison_v15(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1_v1_5'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a

In [44]:
pd.DataFrame(air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono, 'approve_flag', 'fico_pb', asc= False), 'AIR TTD'))

,AIR TTDage_cuts,AIR TTDRace,AIR TTDRaceblack,AIR TTDRacehispanic,AIR TTDGender
0,1.2736,0.7421,0.6415,0.8449,0.9288


In [45]:
pd.DataFrame(air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono[ttd_ono['dataset']== 'Property Owner'], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD'))

,AIR TTDage_cuts,AIR TTDRace,AIR TTDRaceblack,AIR TTDRacehispanic,AIR TTDGender
0,1.1012,0.8121,0.7562,0.8751,0.948


In [46]:
pd.DataFrame(air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono[ttd_ono['dataset']== 'Non Property Owner'], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD'))

,AIR TTDage_cuts,AIR TTDRace,AIR TTDRaceblack,AIR TTDRacehispanic,AIR TTDGender
0,1.291,0.7927,0.6016,0.9756,0.9857


In [47]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['Property Owner'], 'Property Owner', 'Property Owner')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['Property Owner'], 'Overall', 'Property Owner')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['Property Owner'], 'V1.5', 'Property Owner'))]
)

{'segment': ['Property Owner']}
{'segment': ['Property Owner']}
{'segment': ['Property Owner']}


,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,Property Owner,Property Owner,35.14,34.13,33.74,33.36,28.01,40.24,22.57,0.005851,0.005831,0.005398,1.0414,0.8407,0.8785,0.7981,0.9797,1.1012,0.8121,0.7562,0.8751,0.948
0,Overall,Property Owner,28.68,29.96,27.04,27.96,22.62,40.24,22.57,0.005851,0.005831,0.005398,1.1435,0.9376,0.9724,0.8983,0.9969,1.1012,0.8121,0.7562,0.8751,0.948
0,V1.5,Property Owner,37.16,41.03,35.63,38.27,31.60,40.24,22.57,0.005851,0.005831,0.005398,1.0680,0.8210,0.7970,0.8479,0.9637,1.1012,0.8121,0.7562,0.8751,0.948


In [48]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['Non Property Owner'], 'Non Property Owner', 'Non Property Owner')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['Non Property Owner'], 'Overall', 'Non Property Owner')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['Non Property Owner'], 'V1.5', 'Non Property Owner'))]
)

{'segment': ['Non Property Owner']}
{'segment': ['Non Property Owner']}
{'segment': ['Non Property Owner']}


,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,Non Property Owner,Non Property Owner,30.42,28.04,28.64,26.18,26.08,35.69,21.07,0.026538,0.026789,0.025522,1.4264,0.7826,0.7891,0.7762,1.0677,1.291,0.7927,0.6016,0.9756,0.9857
0,Overall,Non Property Owner,24.85,24.70,24.15,21.42,25.75,35.69,21.07,0.026538,0.026789,0.025522,1.5234,0.8922,0.8531,0.9298,0.9969,1.291,0.7927,0.6016,0.9756,0.9857
0,V1.5,Non Property Owner,38.09,36.11,37.36,35.14,34.65,35.69,21.07,0.026538,0.026789,0.025522,1.4140,0.7703,0.6504,0.8848,1.0296,1.291,0.7927,0.6016,0.9756,0.9857


In [49]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['Non Property Owner', 'Property Owner'], 'Both', 'Both')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['Non Property Owner', 'Property Owner'], 'Overall', 'Both')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['Non Property Owner', 'Property Owner'], 'V1.5', 'Both'))]
)

{'segment': ['Non Property Owner', 'Property Owner']}
{'segment': ['Non Property Owner', 'Property Owner']}
{'segment': ['Non Property Owner', 'Property Owner']}


,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,Both,Both,41.26,40.70,41.28,40.74,40.52,44.83,36.64,0.013582,0.013663,0.012948,1.3914,0.6839,0.7322,0.6345,0.9389,1.2736,0.7421,0.6415,0.8449,0.9288
0,Overall,Both,38.51,38.61,38.72,39.42,39.35,44.83,36.64,0.013582,0.013663,0.012948,1.4216,0.7682,0.7737,0.7625,0.9275,1.2736,0.7421,0.6415,0.8449,0.9288
0,V1.5,Both,46.06,45.57,45.54,45.16,43.58,44.83,36.64,0.013582,0.013663,0.012948,1.3940,0.7109,0.6767,0.7460,0.9375,1.2736,0.7421,0.6415,0.8449,0.9288


### END

In [29]:
def get_results(data, data_ttd, target, score, cols_tuple, weight, dataset, asc= False):
    pred= data.copy()
    a= {}
    
    a['dataset']= dataset
    a['model']= score
    a['weight']= weight
    
    t= get_risk_rank(data, target, cols_tuple, score, weight, asc)
    
    a['KS'], a['GINI']= t[1]['KS'], t[1]['Gini']
    
    pred = Equal_Approval_Cutoff_Barsath(data_ttd, 'approve_flag', score)
    air_ttd = air_calc_new(pred,'AIR ttd ')
    a.update({k: v[0] for k, v in air_ttd.items()})
    # print(a)
    
    return a

In [30]:
# get_results(train_ono[train_ono['dataset']== 'Property Owner'], ttd_ono[ttd_ono['dataset']== 'Property Owner'], 'bad_flag', 'p1', col_tuple_pass, 'conwt')

In [31]:
test_ono.head()

,unique_id,client,wt2,fico_pb,auto_score,approve_flag,funded_flag,bad_flag,Race,Gender,age_cuts,p1,dataset,p1_overall_lda
0,264031,Arkansas,1.409924,731.0,732.0,1.0,1.0,0.0,missing,M,Age below 62,0.009460,Property Owner,0.017647
1,2019-07-31 14:42|528-29-0782,GESA,1.328745,811.0,804.0,1.0,1.0,0.0,missing,M,Age below 62,0.002785,Property Owner,0.005781
2,TCI275526,Numerica,1.194283,782.0,742.0,1.0,1.0,0.0,white,F,Age 62 and over,0.009691,Property Owner,0.006465
3,366242,Tower,1.290555,681.0,784.0,1.0,1.0,0.0,white,I,Age below 62,0.006195,Property Owner,0.006465
4,279088,Arkansas,1.281995,725.0,803.0,1.0,1.0,0.0,missing,F,Age below 62,0.004505,Property Owner,0.006115


In [54]:
print('Property Owner')
print('Bad Rate Train', train_ono[train_ono['dataset']== 'Property Owner']['bad_flag'].mean())
print('Bad Rate Test', test_ono[test_ono['dataset']== 'Property Owner']['bad_flag'].mean())
print('Bad Rate Oot', oot_ono[oot_ono['dataset']== 'Property Owner']['bad_flag'].mean())

print('KS Weighted Train p1', rnkOrderingN(train_ono[train_ono['dataset']== 'Property Owner'], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Weighted Test p1', rnkOrderingN(test_ono[test_ono['dataset']== 'Property Owner'], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS'])
      
print('KS Non Weighted Train p1', rnkOrderingN(train_ono[train_ono['dataset']== 'Property Owner'], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Test p1', rnkOrderingN(test_ono[test_ono['dataset']== 'Property Owner'], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Oot p1', rnkOrderingN(oot_ono[oot_ono['dataset']== 'Property Owner'], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])

print('KS Non Weighted Test FICO', rnkOrderingN(test_ono[test_ono['dataset']== 'Property Owner'], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS'])
print('KS Non Weighted Test Auto Score', rnkOrderingN(test_ono[test_ono['dataset']== 'Property Owner'], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS'])

print('AIR Table', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono[ttd_ono['dataset']== 'Property Owner'], 'approve_flag', 'p1'), 'AIR TTD'))

print('KS Weighted Train p1 overall', rnkOrderingN(train_ono[train_ono['dataset']== 'Property Owner'], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Weighted Test p1 overall', rnkOrderingN(test_ono[test_ono['dataset']== 'Property Owner'], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
      
print('KS Non Weighted Train p1 overall', rnkOrderingN(train_ono[train_ono['dataset']== 'Property Owner'], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Test p1 overall', rnkOrderingN(test_ono[test_ono['dataset']== 'Property Owner'], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Oot p1 overall', rnkOrderingN(oot_ono[oot_ono['dataset']== 'Property Owner'], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])

print('AIR Table', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono[ttd_ono['dataset']== 'Property Owner'], 'approve_flag', 'p1_overall_lda'), 'AIR TTD'))
print('AIR Table FICO', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono[ttd_ono['dataset']== 'Property Owner'], 'approve_flag', 'fico_pb', False), 'AIR TTD'))

Property Owner
Bad Rate Train 0.0058513037436153745
Bad Rate Test 0.005830820865805791
Bad Rate Oot 0.005397590361445783
KS Weighted Train p1 35.14
KS Weighted Test p1 34.13
KS Non Weighted Train p1 33.74
KS Non Weighted Test p1 33.36
KS Non Weighted Oot p1 28.01
KS Non Weighted Test FICO 40.24
KS Non Weighted Test Auto Score 22.57
AIR Table {'AIR TTDage_cuts': [1.0414], 'AIR TTDRace': [0.8407], 'AIR TTDRaceblack': [0.8785], 'AIR TTDRacehispanic': [0.7981], 'AIR TTDGender': [0.9797]}
KS Weighted Train p1 overall 27.15
KS Weighted Test p1 overall 32.7
KS Non Weighted Train p1 overall 24.93
KS Non Weighted Test p1 overall 27.96
KS Non Weighted Oot p1 overall 19.93
AIR Table {'AIR TTDage_cuts': [1.0992], 'AIR TTDRace': [0.9242], 'AIR TTDRaceblack': [0.9449], 'AIR TTDRacehispanic': [0.901], 'AIR TTDGender': [0.9922]}
AIR Table FICO {'AIR TTDage_cuts': [1.1012], 'AIR TTDRace': [0.8121], 'AIR TTDRaceblack': [0.7562], 'AIR TTDRacehispanic': [0.8751], 'AIR TTDGender': [0.948]}


In [55]:
print('Non Property Owner')
print('Bad Rate Train', train_ono[train_ono['dataset']== 'Non Property Owner']['bad_flag'].mean())
print('Bad Rate Test', test_ono[test_ono['dataset']== 'Non Property Owner']['bad_flag'].mean())
print('Bad Rate Oot', oot_ono[oot_ono['dataset']== 'Non Property Owner']['bad_flag'].mean())

print('KS Weighted Train p1', rnkOrderingN(train_ono[train_ono['dataset']== 'Non Property Owner'], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Weighted Test p1', rnkOrderingN(test_ono[test_ono['dataset']== 'Non Property Owner'], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS'])
      
print('KS Non Weighted Train p1', rnkOrderingN(train_ono[train_ono['dataset']== 'Non Property Owner'], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Test p1', rnkOrderingN(test_ono[test_ono['dataset']== 'Non Property Owner'], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Oot p1', rnkOrderingN(oot_ono[oot_ono['dataset']== 'Non Property Owner'], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])

print('KS Non Weighted Test FICO', rnkOrderingN(test_ono[test_ono['dataset']== 'Non Property Owner'], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS'])
print('KS Non Weighted Test Auto Score', rnkOrderingN(test_ono[test_ono['dataset']== 'Non Property Owner'], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS'])

print('AIR Table', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono[ttd_ono['dataset']== 'Non Property Owner'], 'approve_flag', 'p1'), 'AIR TTD'))

print('KS Weighted Train p1 overall', rnkOrderingN(train_ono[train_ono['dataset']== 'Non Property Owner'], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Weighted Test p1 overall', rnkOrderingN(test_ono[test_ono['dataset']== 'Non Property Owner'], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
      
print('KS Non Weighted Train p1 overall', rnkOrderingN(train_ono[train_ono['dataset']== 'Non Property Owner'], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Test p1 overall', rnkOrderingN(test_ono[test_ono['dataset']== 'Non Property Owner'], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Oot p1 overall', rnkOrderingN(oot_ono[oot_ono['dataset']== 'Non Property Owner'], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])

print('AIR Table', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono[ttd_ono['dataset']== 'Non Property Owner'], 'approve_flag', 'p1_overall_lda'), 'AIR TTD'))
print('AIR Table FICO', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono[ttd_ono['dataset']== 'Non Property Owner'], 'approve_flag', 'fico_pb', False), 'AIR TTD'))

Non Property Owner
Bad Rate Train 0.026537825083249915
Bad Rate Test 0.026788693455360123
Bad Rate Oot 0.025521669341894062
KS Weighted Train p1 30.42
KS Weighted Test p1 28.04
KS Non Weighted Train p1 28.64
KS Non Weighted Test p1 26.18
KS Non Weighted Oot p1 26.08
KS Non Weighted Test FICO 35.69
KS Non Weighted Test Auto Score 21.07
AIR Table {'AIR TTDage_cuts': [1.4264], 'AIR TTDRace': [0.7826], 'AIR TTDRaceblack': [0.7891], 'AIR TTDRacehispanic': [0.7762], 'AIR TTDGender': [1.0677]}
KS Weighted Train p1 overall 26.12
KS Weighted Test p1 overall 23.86
KS Non Weighted Train p1 overall 25.09
KS Non Weighted Test p1 overall 20.41
KS Non Weighted Oot p1 overall 24.65
AIR Table {'AIR TTDage_cuts': [1.4657], 'AIR TTDRace': [0.8544], 'AIR TTDRaceblack': [0.7841], 'AIR TTDRacehispanic': [0.9215], 'AIR TTDGender': [1.0017]}
AIR Table FICO {'AIR TTDage_cuts': [1.291], 'AIR TTDRace': [0.7927], 'AIR TTDRaceblack': [0.6016], 'AIR TTDRacehispanic': [0.9756], 'AIR TTDGender': [0.9857]}


In [56]:
train_ono['dataset'].value_counts(dropna= False)

dataset
Property Owner        82033
Non Property Owner    48949
Name: count, dtype: int64

In [57]:
print('Overall')
print('Bad Rate Train', train_ono['bad_flag'].mean())
print('Bad Rate Test', test_ono['bad_flag'].mean())
print('Bad Rate Oot', oot_ono['bad_flag'].mean())

print('KS Weighted Train p1', rnkOrderingN(train_ono, 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Weighted Test p1', rnkOrderingN(test_ono, 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS'])
      
print('KS Non Weighted Train p1', rnkOrderingN(train_ono, 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Test p1', rnkOrderingN(test_ono, 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Oot p1', rnkOrderingN(oot_ono, 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS'])

print('KS Non Weighted Test FICO', rnkOrderingN(test_ono, 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS'])
print('KS Non Weighted Test Auto Score', rnkOrderingN(test_ono, 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS'])

print('AIR Table', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono, 'approve_flag', 'p1'), 'AIR TTD'))

print('KS Weighted Train p1 overall', rnkOrderingN(train_ono, 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Weighted Test p1 overall', rnkOrderingN(test_ono, 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
      
print('KS Non Weighted Train p1 overall', rnkOrderingN(train_ono, 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Test p1 overall', rnkOrderingN(test_ono, 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])
print('KS Non Weighted Oot p1 overall', rnkOrderingN(oot_ono, 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS'])

print('AIR Table', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono, 'approve_flag', 'p1_overall_lda'), 'AIR TTD'))
print('AIR Table FICO', air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono, 'approve_flag', 'fico_pb', False), 'AIR TTD'))

Overall
Bad Rate Train 0.013582018903360768
Bad Rate Test 0.013663003010492189
Bad Rate Oot 0.012947907256850346
KS Weighted Train p1 41.26
KS Weighted Test p1 40.7
KS Non Weighted Train p1 41.28
KS Non Weighted Test p1 40.74
KS Non Weighted Oot p1 40.52
KS Non Weighted Test FICO 44.83
KS Non Weighted Test Auto Score 36.64
AIR Table {'AIR TTDage_cuts': [1.3914], 'AIR TTDRace': [0.6839], 'AIR TTDRaceblack': [0.7322], 'AIR TTDRacehispanic': [0.6345], 'AIR TTDGender': [0.9389]}
KS Weighted Train p1 overall 39.78
KS Weighted Test p1 overall 40.08
KS Non Weighted Train p1 overall 40.02
KS Non Weighted Test p1 overall 40.87
KS Non Weighted Oot p1 overall 39.11
AIR Table {'AIR TTDage_cuts': [1.4272], 'AIR TTDRace': [0.7423], 'AIR TTDRaceblack': [0.7657], 'AIR TTDRacehispanic': [0.7185], 'AIR TTDGender': [0.9194]}
AIR Table FICO {'AIR TTDage_cuts': [1.2736], 'AIR TTDRace': [0.7421], 'AIR TTDRaceblack': [0.6415], 'AIR TTDRacehispanic': [0.8449], 'AIR TTDGender': [0.9288]}


In [36]:
pw= train_ono[train_ono['dataset']== 'Property Owner'].reset_index(drop= True)
pw.shape

(82033, 15)

In [37]:
npw= train_ono[train_ono['dataset']== 'Non Property Owner'].reset_index(drop= True)
npw.shape

(48949, 15)

In [50]:
pw['bad_flag'].mean()*100, npw['bad_flag'].mean()*100, npw['bad_flag'].sum(), pw['bad_flag'].sum()

(0.5851303743615375, 2.6537825083249915, 1299.0, 480.0)

In [51]:
pw_good= pw[pw['bad_flag']!= 1]
pw_bad= pw[pw['bad_flag']== 1]

print(pw_good.shape, pw_bad.shape)

pw_good= pw_good.sample(frac= 0.5, random_state= 101)
print(pw_good.shape)

pw= pd.concat([pw_good, pw_bad]).reset_index(drop= True)
print(pw.shape, pw['bad_flag'].sum(), pw['bad_flag'].mean())

(81553, 15) (480, 15)
(40776, 15)
(41256, 15) 480.0 0.011634671320535195


In [52]:
npw_good= npw[npw['bad_flag']!= 1]
npw_bad= npw[npw['bad_flag']== 1]

print(npw_good.shape, npw_bad.shape)

npw_good= npw_good.sample(frac= 0.85, random_state= 101)
print(npw_good.shape)

npw_bad= npw_bad.sample(frac= 0.38, random_state= 101)
print(npw_bad.shape)

npw= pd.concat([npw_good, npw_bad]).reset_index(drop= True)
print(npw.shape, npw['bad_flag'].sum(), npw['bad_flag'].mean())

(47650, 15) (1299, 15)
(40502, 15)
(494, 15)
(40996, 15) 494.0 0.012049956093277393


In [53]:
print(train_ono.shape, train_ono['bad_flag'].sum(), train_ono['bad_flag'].mean())
train_ono= pd.concat([pw, npw]).reset_index(drop= True)
print(train_ono.shape, train_ono['bad_flag'].sum(), train_ono['bad_flag'].mean())

(130982, 15) 1779.0 0.013582018903360768
(82252, 15) 974.0 0.011841657345718038
